## 1. Libraries and Folders

In [1]:
path="C:/Users/bvarghese/Desktop/IESEG Slides/Financial Programming- Python- Phan Minh/Group Project_Customer Loan"

In [2]:
pip install pandasql

Note: you may need to restart the kernel to use updated packages.


In [3]:
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

In [4]:
pip install --upgrade kmodes


Requirement already up-to-date: kmodes in c:\users\bvarghese\appdata\local\continuum\anaconda3\lib\site-packages (0.10.1)
Note: you may need to restart the kernel to use updated packages.


In [5]:
#External Libraries added
import sys

sys.path.append(path)

import numpy                as np
import pandas               as pd
import seaborn              as sns
from pandas                 import DataFrame
from pandas                 import read_csv
from pandas                 import pivot_table
from pandas                 import Series
from pandas                 import get_dummies
from numpy                  import random
from numpy                  import where
from numpy                  import nan
from numpy                  import array
from sklearn.ensemble       import RandomForestClassifier
from sklearn.ensemble       import GradientBoostingClassifier
from sklearn.metrics        import accuracy_score
from sklearn.metrics        import roc_auc_score
from sklearn.metrics        import auc
from sklearn.metrics        import recall_score
from sklearn.cluster        import KMeans
from sklearn.tree           import DecisionTreeClassifier
from sklearn.linear_model   import LogisticRegression 
from sklearn.svm            import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors      import KNeighborsClassifier
from sklearn                import preprocessing
from matplotlib             import pyplot
from scipy.stats.mstats     import winsorize #we dont used it
from scipy.stats            import pearsonr
from matplotlib             import pyplot
from ieseg                  import partition
from ieseg                  import roc
from ieseg                  import lift
from ieseg                  import cumulativeResponse
from ieseg                  import cumulativeGains
from imblearn.over_sampling import SMOTE
from sklearn.cluster        import AgglomerativeClustering
from mpl_toolkits.mplot3d   import Axes3D 
from kmodes.kmodes          import KModes

## 2. Data Preparation

In [6]:
#Paths

directory = path
inputData = directory + "/Data"
sandbox   = directory + "/Sandbox"

In [7]:
# Functions used in the notebook 

def explore(data):
    print(data.head())
    print(data.tail())
    print(data.info())
    print(data.columns)
    print(data.dtypes)
    print(data.shape)
    print(data.describe())

def trans_type(x):
    if x == 'PRIJEM':
        return 'Credit'
    elif x == 'VYDAJ':
        return 'Withdrawl'
    else:
        return 'Withdrawl'
    
def trans_operation(x):
    if x == 'VYBER KARTOU':
        return 'Credit_card_withdrawl'
    elif x == 'VKLAD':
        return 'Credit_in_cash'
    elif x == 'PREVOD Z UCTU':
        return 'Collect_other_bank'
    elif x == 'VYBER':
        return 'Cash_withdrawl'
    elif x == 'PREVOD NA UCET':
        return 'Remit_other_bank'    
    else:
        return 'Other_Operations'
    
def trans_character(x):
    if x == 'POJISTNE':
        return 'Insurance_Payment'
    elif x == 'SLUZBY':
        return 'Statement_Payment'
    elif x == 'UROK':
        return 'Credited_Interest'
    elif x == 'SANKC. UROK':
        return 'Sanction_Interest'
    elif x == 'SIPO':
        return 'Household'
    elif x == 'DUCHOD':
        return 'Old_Age_Pension'  
    elif x == 'UVER':
        return 'Loan_Payment'      
    else:
        return 'Other_Payments'
    
def account_freq(x):
    if x == 'POPLATEK MESICNE':
        return 'Monthly_Issuance'
    elif x == 'POPLATEK TYDNE':
        return 'Weekly_Issuance'
    else:
        return 'After_trans_Issuance'
    
def orders_payment(x):
    if x == 'POJISTNE':
        return 'Insurance_payment'
    elif x == 'SIPO':
        return 'Household_Payment'
    elif x == 'LEASING':
        return 'leasing'
    elif x == 'UVER':
        return 'Loan_Payment'
    else:
        return 'Other_Order_Payment'
    

## 3. Importing Data

In [8]:
# Note: This data was extracted on 1999

# Each record describes characteristics of a client
client = pd.read_csv('C:/Users/bvarghese/Desktop/IESEG Slides/Financial Programming- Python- Phan Minh/Group Project_Customer Loan/Data/client.asc', sep=';')

# Each record describes static characteristics of an account
account = pd.read_csv('C:/Users/bvarghese/Desktop/IESEG Slides/Financial Programming- Python- Phan Minh/Group Project_Customer Loan/Data/account.asc', sep=';')

# Each record describes a credit card issued to an account
card = pd.read_csv('C:/Users/bvarghese/Desktop/IESEG Slides/Financial Programming- Python- Phan Minh/Group Project_Customer Loan/Data/card.asc', sep=';')

# Each record describes demographic characteristics of a district
district = pd.read_csv('C:/Users/bvarghese/Desktop/IESEG Slides/Financial Programming- Python- Phan Minh/Group Project_Customer Loan/Data/district.asc', sep=';')

# Each record relates together a client with an account
# i.e. this relation describes the rights of clients to operate accounts
disp = pd.read_csv('C:/Users/bvarghese/Desktop/IESEG Slides/Financial Programming- Python- Phan Minh/Group Project_Customer Loan/Data/disp.asc', sep=';')

# Each record describes characteristics of a payment order (debits only)
order = pd.read_csv('C:/Users/bvarghese/Desktop/IESEG Slides/Financial Programming- Python- Phan Minh/Group Project_Customer Loan/Data/order.asc', sep=';')

# Each record describes one transaction on an account
trans = pd.read_csv('C:/Users/bvarghese/Desktop/IESEG Slides/Financial Programming- Python- Phan Minh/Group Project_Customer Loan/Data/trans.asc', sep=';', low_memory=False)

# Each record describes a loan granted for a given account
loan = pd.read_csv('C:/Users/bvarghese/Desktop/IESEG Slides/Financial Programming- Python- Phan Minh/Group Project_Customer Loan/Data/loan.asc', sep=';')

## 4. Transactions Data

### Inspecting the data

In [9]:
explore(trans)

   trans_id  account_id    date    type operation  amount  balance k_symbol  \
0    695247        2378  930101  PRIJEM     VKLAD   700.0    700.0      NaN   
1    171812         576  930101  PRIJEM     VKLAD   900.0    900.0      NaN   
2    207264         704  930101  PRIJEM     VKLAD  1000.0   1000.0      NaN   
3   1117247        3818  930101  PRIJEM     VKLAD   600.0    600.0      NaN   
4    579373        1972  930102  PRIJEM     VKLAD   400.0    400.0      NaN   

  bank  account  
0  NaN      NaN  
1  NaN      NaN  
2  NaN      NaN  
3  NaN      NaN  
4  NaN      NaN  
         trans_id  account_id    date    type operation  amount  balance  \
1056315   3626622        2906  981231  PRIJEM       NaN    62.3  13729.4   
1056316   3627616        2935  981231  PRIJEM       NaN    81.3  19544.9   
1056317   3625403        2869  981231  PRIJEM       NaN    60.2  14638.2   
1056318   3626683        2907  981231  PRIJEM       NaN   107.5  23453.0   
1056319   3626540        2902  981231

In [10]:
# Check missing values

print(trans.isna().sum().sum())

for column in trans.columns:
    print(column, ':', trans[column].isna().sum())

2208738
trans_id : 0
account_id : 0
date : 0
type : 0
operation : 183114
amount : 0
balance : 0
k_symbol : 481881
bank : 782812
account : 760931


### Modififying observations and creating new variables

In [11]:
# Changing observations from codes to text

trans['type'] = trans['type'].apply(lambda row:trans_type(row))
trans['operation'] = trans['operation'].apply(lambda row:trans_operation(row))
trans['k_symbol'] = trans['k_symbol'].apply(lambda row:trans_character(row))

# Change date to a readable format

trans['date'] = trans['date'].transform(lambda bn: int('19' + str(bn)))
trans['date'] = trans['date'].apply(lambda x: pd.to_datetime(x, format='%Y%m%d'))

trans.head()

,trans_id,account_id,date,type,operation,amount,balance,k_symbol,bank,account
0,695247,2378,1993-01-01,Credit,Credit_in_cash,700.0,700.0,Other_Payments,NaN,NaN
1,171812,576,1993-01-01,Credit,Credit_in_cash,900.0,900.0,Other_Payments,NaN,NaN
2,207264,704,1993-01-01,Credit,Credit_in_cash,1000.0,1000.0,Other_Payments,NaN,NaN
3,1117247,3818,1993-01-01,Credit,Credit_in_cash,600.0,600.0,Other_Payments,NaN,NaN
4,579373,1972,1993-01-02,Credit,Credit_in_cash,400.0,400.0,Other_Payments,NaN,NaN


### Variables representing aggregations using amount, operation, k_symbol

In [12]:
# Find the first and last transaction date for each account_id
trans_date = trans.groupby('account_id')['date'].agg(['min', 'max', 'count'])

# Create variables to find totals for each account_id
trans_amount = trans.groupby('account_id')['amount'].agg(['sum', 'mean'])

# Merge the two tables
trans_cal = pd.merge(trans_date, trans_amount, 'left', on = 'account_id')

# View the results
trans_cal.head()

,min,max,count,sum,mean
account_id,,,,,
1,1995-03-24,1998-12-31,239,375174.5,1569.767782
2,1993-02-26,1998-12-31,478,3151479.3,6593.052929
3,1997-07-07,1998-12-31,117,295021.8,2521.553846
4,1996-02-21,1998-12-31,186,350971.4,1886.943011
5,1997-05-30,1998-12-31,84,166881.4,1986.683333


In [13]:
# Use mathematical metrics on the variable - operations and k_symbol
trans_add = trans.groupby(['account_id', 'operation', 'k_symbol'])['amount'].agg(['count', 'sum', 'mean'])
trans_add2 = trans.groupby(['account_id', 'k_symbol'])['amount'].agg(['count', 'sum', 'mean'])

# Pivot the two tables
trans_add_pivot = trans_add.pivot_table(index = 'account_id', columns = 'operation', values = ['count', 'sum', 'mean'])
trans_add_pivot2 = trans_add2.pivot_table(index = 'account_id', columns = 'k_symbol', values = ['count', 'sum', 'mean'])

# Merge the two pivoted tables
trans_add_merge = pd.merge(trans_add_pivot, trans_add_pivot2, 'left', left_index = True, right_index= True)
trans_cal2 = pd.merge(trans_cal, trans_add_merge, 'left', on = 'account_id')

# View the results
trans_cal2.head()


C:\Users\bvarghese\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\reshape\merge.py:617: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


,min,max,count,sum,mean,"(count, Cash_withdrawl)","(count, Collect_other_bank)","(count, Credit_card_withdrawl)","(count, Credit_in_cash)","(count, Other_Operations)",...,"(mean, Sanction_Interest)","(mean, Statement_Payment)","(sum, Credited_Interest)","(sum, Household)","(sum, Insurance_Payment)","(sum, Loan_Payment)","(sum, Old_Age_Pension)","(sum, Other_Payments)","(sum, Sanction_Interest)","(sum, Statement_Payment)"
account_id,,,,,,,,,,,,,,,,,,,,,
1,1995-03-24,1998-12-31,239,375174.5,1569.767782,48.5,45.0,NaN,12.0,45.0,...,NaN,14.6,3065.5,98080.0,NaN,NaN,NaN,273445.0,NaN,584.0
2,1993-02-26,1998-12-31,478,3151479.3,6593.052929,118.5,70.0,NaN,12.0,70.0,...,NaN,14.6,10717.5,472290.0,NaN,80944.8,NaN,2586578.0,NaN,949.0
3,1997-07-07,1998-12-31,117,295021.8,2521.553846,16.0,NaN,NaN,28.0,18.0,...,NaN,14.6,2070.0,14755.0,46007.0,NaN,NaN,232000.0,NaN,189.8
4,1996-02-21,1998-12-31,186,350971.4,1886.943011,31.0,34.0,NaN,1.0,30.0,...,NaN,14.6,2744.0,60262.0,NaN,NaN,188802.0,98740.0,NaN,423.4
5,1997-05-30,1998-12-31,84,166881.4,1986.683333,17.5,19.0,NaN,1.0,15.0,...,NaN,14.6,1562.0,37352.0,NaN,NaN,95323.0,32440.0,NaN,204.4


In [14]:
# Change the column names 

trans_cal2.rename(columns = {'min' : 'first_trans'                                             , 'max' : 'last_trans',
                             'count' : 'total_trans'                                           , 'sum' : 'total_sum_trans',
                             'mean' : 'mean_all_trans'                                         , ('count', 'Cash_withdrawl') : 'num_cash_withdrawl',
                             ('count', 'Collect_other_bank') : 'num_trans_collections'         , ('count', 'Credit_card_withdrawl') : 'num_trans_cc_withdrawls',
                             ('count', 'Credit_in_cash') : 'num_trans_credit_cash'             , ('count', 'Other_Operations') : 'num_trans_Other_Operations',
                             ('count', 'Remit_other_bank') : 'num_trans_remittances'           , ('mean', 'Cash_withdrawl') : 'avg_trans_cash_withdrawl',
                             ('mean', 'Collect_other_bank') : 'avg_trans_collections'          , ('mean', 'Credit_card_withdrawl') :  'avg_trans_cc_withdrawl',
                             ('mean', 'Credit_in_cash') : 'avg_trans_credit_in_cash'           , ('mean', 'Other_Operations') : 'avg_trans_Other_Operations',
                             ('mean', 'Remit_other_bank') : 'avg_trans_remittances'            , ('sum', 'Cash_withdrawl') : 'total_trans_cash_withdrawl',
                             ('sum', 'Collect_other_bank') : 'total_trans_collections'         , ('sum', 'Credit_card_withdrawl') : 'total_trans_cc_withdrawl',
                             ('sum', 'Credit_in_cash') : 'total_trans_credit_in_cash'          , ('sum', 'Other_Operations') : 'total_trans_Other_Operations',
                             ('sum', 'Remit_other_bank'): 'total_trans_remittances'            , ('count', 'Credited_Interest') : 'num_trans_credited_interest',
                             ('count', 'Household') : 'num_trans_household'                    , ('count', 'Insurance_Payment') : 'num_trans_insurance_payment',
                             ('count', 'Loan_Payment') : 'num_trans_loan_payment'              , ('count', 'Other_Payments') : 'num_trans_Other_Payments',
                             ('count', 'Old_Age_Pension') : 'num_trans_old_age_pensions'       , ('count', 'Sanction_Interest') : 'num_trans_sanction_interest',
                             ('count', 'Statement_Payment') : 'num_trans_statement_payment'    , ('mean', 'Credited_Interest') : 'avg_trans_credited_interest',
                             ('mean', 'Household') : 'avg_trans_household'                     , ('mean', 'Insurance_Payment') : 'avg_trans_insurance_payment',
                             ('mean', 'Loan_Payment') : 'avg_trans_loan_payment'               , ('mean', 'Other_Payments') : 'avg_trans_Other_Payments',
                             ('mean', 'Old_Age_Pension') : 'avg_trans_old_age_pension'         , ('mean', 'Sanction_Interest') : 'avg_trans_sanction_interest',
                             ('mean', 'Statement_Payment') : 'mean_trans_statement_payment'    , ('sum', 'Credited_Interest') : 'total_trans_credited_interest',
                             ('sum', 'Household') : 'total_trans_households'                   , ('sum', 'Insurance_Payment') : 'total_trans_insurance_payment',
                             ('sum', 'Loan_Payment') : 'total_trans_loan_payment'              , ('sum', 'Other_Payments') : 'total_trans_Other_Payments',
                             ('sum', 'Old_Age_Pension') : 'total_trans_old_age_pensions'       , ('sum', 'Sanction_Interest') : 'total_trans_sanction_interest',
                             ('sum', 'Statement_Payment') : 'total_trans_statement_payment'}   , inplace = True)


In [15]:
# Replace the NAs with 0

print(trans_cal2.isna().sum().sum())
trans_cal2 = trans_cal2.fillna(0)

74070


In [16]:
#Create the final transaction table
trans_final = trans_cal2

#Add a column to evaluate the number of days since the last transaction (Assuming this was evaluated on 1 January, 1999)
date_1 = pd.to_datetime('1999-01-01')
trans_final['days_since_last_trans'] = (date_1 - trans_final['last_trans']) / np.timedelta64(1, 'D')
trans_final['days_since_last_trans'] = trans_final['days_since_last_trans'].astype(int)
trans_final.head()

,first_trans,last_trans,total_trans,total_sum_trans,mean_all_trans,num_cash_withdrawl,num_trans_collections,num_trans_cc_withdrawls,num_trans_credit_cash,num_trans_Other_Operations,...,mean_trans_statement_payment,total_trans_credited_interest,total_trans_households,total_trans_insurance_payment,total_trans_loan_payment,total_trans_old_age_pensions,total_trans_Other_Payments,total_trans_sanction_interest,total_trans_statement_payment,days_since_last_trans
account_id,,,,,,,,,,,,,,,,,,,,,
1,1995-03-24,1998-12-31,239,375174.5,1569.767782,48.5,45.0,0.0,12.0,45.0,...,14.6,3065.5,98080.0,0.0,0.0,0.0,273445.0,0.0,584.0,1
2,1993-02-26,1998-12-31,478,3151479.3,6593.052929,118.5,70.0,0.0,12.0,70.0,...,14.6,10717.5,472290.0,0.0,80944.8,0.0,2586578.0,0.0,949.0,1
3,1997-07-07,1998-12-31,117,295021.8,2521.553846,16.0,0.0,0.0,28.0,18.0,...,14.6,2070.0,14755.0,46007.0,0.0,0.0,232000.0,0.0,189.8,1
4,1996-02-21,1998-12-31,186,350971.4,1886.943011,31.0,34.0,0.0,1.0,30.0,...,14.6,2744.0,60262.0,0.0,0.0,188802.0,98740.0,0.0,423.4,1
5,1997-05-30,1998-12-31,84,166881.4,1986.683333,17.5,19.0,0.0,1.0,15.0,...,14.6,1562.0,37352.0,0.0,0.0,95323.0,32440.0,0.0,204.4,1


## 5. Credit Card Data

### Inspecting the data

In [17]:
explore(card)

   card_id  disp_id     type           issued
0     1005     9285  classic  931107 00:00:00
1      104      588  classic  940119 00:00:00
2      747     4915  classic  940205 00:00:00
3       70      439  classic  940208 00:00:00
4      577     3687  classic  940215 00:00:00
     card_id  disp_id     type           issued
887      125      694     gold  981226 00:00:00
888      674     4360  classic  981228 00:00:00
889      322     2063  classic  981228 00:00:00
890      685     4467  classic  981229 00:00:00
891      635     4083  classic  981229 00:00:00
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 892 entries, 0 to 891
Data columns (total 4 columns):
card_id    892 non-null int64
disp_id    892 non-null int64
type       892 non-null object
issued     892 non-null object
dtypes: int64(2), object(2)
memory usage: 28.0+ KB
None
Index(['card_id', 'disp_id', 'type', 'issued'], dtype='object')
card_id     int64
disp_id     int64
type       object
issued     object
dtype: object
(892

In [18]:
# Checking for missing values
card.isna().sum().sum()

0

In [19]:
# Change date to a readable format

card['issued'] = card['issued'].transform(lambda bn: ('19' + bn))
card['issued'] = card['issued'].apply(lambda x: pd.to_datetime(x, format='%Y%m%d %H:%M:%S'))
card.head()

,card_id,disp_id,type,issued
0,1005,9285,classic,1993-11-07
1,104,588,classic,1994-01-19
2,747,4915,classic,1994-02-05
3,70,439,classic,1994-02-08
4,577,3687,classic,1994-02-15


## 6. Account Data

### Inspecting the data

In [20]:
explore(account)

   account_id  district_id         frequency    date
0         576           55  POPLATEK MESICNE  930101
1        3818           74  POPLATEK MESICNE  930101
2         704           55  POPLATEK MESICNE  930101
3        2378           16  POPLATEK MESICNE  930101
4        2632           24  POPLATEK MESICNE  930102
      account_id  district_id         frequency    date
4495         124           55  POPLATEK MESICNE  971228
4496        3958           59  POPLATEK MESICNE  971228
4497         777           30  POPLATEK MESICNE  971228
4498        1573           63  POPLATEK MESICNE  971229
4499        3276            1  POPLATEK MESICNE  971229
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4500 entries, 0 to 4499
Data columns (total 4 columns):
account_id     4500 non-null int64
district_id    4500 non-null int64
frequency      4500 non-null object
date           4500 non-null int64
dtypes: int64(3), object(1)
memory usage: 140.8+ KB
None
Index(['account_id', 'district_id', 'frequ

In [21]:
#Checking for missing values

account.isna().sum().sum()

0

In [22]:
# Change frequency observations into a more interpretable form
account['frequency'] = account['frequency'].apply(lambda row:account_freq(row))

#Assign Dummies to the Frequency Variable
account = pd.get_dummies(account, columns = ['frequency'])

# Change date to a readable format
account['date'] = account['date'].transform(lambda bn: int('19' + str(bn)))
account['account_creation_date'] = account['date'].apply(lambda x: pd.to_datetime(x, format='%Y%m%d'))

# Drop older date column
account = account.drop(columns = 'date')

#Add a column to evaluate the number of days since the account was created (Assuming this was evaluated on 1 January, 1999)
account['days_since_acc_creation'] = (date_1 - account['account_creation_date'])  / np.timedelta64(1, 'D')
account['days_since_acc_creation'] = account['days_since_acc_creation'].astype(int)

# Sort the observations by the account_id
account.sort_values('account_id')

#Renaming the dummy columns
account.rename(columns = { "frequency_After_trans_Issuance": "Issuance_after_transaction",
                           "frequency_Monthly_Issuance"    : "Monthly_Issuance",
                           "frequency_Weekly_Issuance"     : "Weekly_Issuance"}, inplace= True)

#View the results
account.head()


,account_id,district_id,Issuance_after_transaction,Monthly_Issuance,Weekly_Issuance,account_creation_date,days_since_acc_creation
0,576,55,0,1,0,1993-01-01,2191
1,3818,74,0,1,0,1993-01-01,2191
2,704,55,0,1,0,1993-01-01,2191
3,2378,16,0,1,0,1993-01-01,2191
4,2632,24,0,1,0,1993-01-02,2190


## 7. Order Data

In [23]:
explore(order)

   order_id  account_id bank_to  account_to  amount k_symbol
0     29401           1      YZ    87144583  2452.0     SIPO
1     29402           2      ST    89597016  3372.7     UVER
2     29403           2      QR    13943797  7266.0     SIPO
3     29404           3      WX    83084338  1135.0     SIPO
4     29405           3      CD    24485939   327.0         
      order_id  account_id bank_to  account_to  amount  k_symbol
6466     46334       11362      YZ    70641225  4780.0      SIPO
6467     46335       11362      MN    78507822    56.0          
6468     46336       11362      ST    40799850   330.0  POJISTNE
6469     46337       11362      KL    20009470   129.0          
6470     46338       11362      MN    61540514  5392.0      UVER
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6471 entries, 0 to 6470
Data columns (total 6 columns):
order_id      6471 non-null int64
account_id    6471 non-null int64
bank_to       6471 non-null object
account_to    6471 non-null int64
a

In [24]:
order.isna().sum().sum()

0

In [25]:
print(order[order.k_symbol == 'SIPO'].count().sum())
print(order[order.k_symbol == 'POJISTNE'].count().sum())
print(order[order.k_symbol == 'LEASING'].count().sum())
print(order[order.k_symbol == 'UVER'].count().sum())
print(order[order.k_symbol == ' '].count().sum())

21012
3192
2046
4302
8274


In [26]:
order['k_symbol'] = order['k_symbol'].apply(lambda row: orders_payment(row))
order.head()


,order_id,account_id,bank_to,account_to,amount,k_symbol
0,29401,1,YZ,87144583,2452.0,Household_Payment
1,29402,2,ST,89597016,3372.7,Loan_Payment
2,29403,2,QR,13943797,7266.0,Household_Payment
3,29404,3,WX,83084338,1135.0,Household_Payment
4,29405,3,CD,24485939,327.0,Other_Order_Payment


In [27]:
order_cal = order.groupby('account_id')['amount'].agg(['count', 'sum', 'mean'])
order_cal2 = order.groupby(['account_id','k_symbol'])['amount'].agg(['count', 'sum', 'mean'])

order_cal2_pivot = order_cal2.pivot_table(index = 'account_id', columns = 'k_symbol', values = ['count', 'sum', 'mean'])

order_final = pd.merge(order_cal, order_cal2_pivot, 'left', on = 'account_id')

In [28]:
order_final.rename(columns = {'count' : 'total_nbr_orders'                                      , 'sum' : 'sum_all_orders',
                              'mean' : 'average_all_orders'                                     , ('count', 'Household_Payment') : 'num_orders_house_payment',
                              ('count', 'Insurance_payment') : 'num_orders_insurance_payment'   , ('count', 'Loan_Payment') : 'num_orders_loan_payment',
                              ('count', 'Other_Order_Payment') : 'num_other_orders'             , ('count', 'leasing') : 'num_leasing_payment',
                              ('mean', 'Household_Payment') : 'avg_household_payment'           , ('mean', 'Insurance_payment') : 'avg_insurance_payment',
                              ('mean', 'Loan_Payment') : 'avg_loan_payment'                     , ('mean', 'Other_Order_Payment') : 'avg_other_orders',
                              ('mean', 'leasing') : 'avg_leasing_payment'                       , ('sum', 'Household_Payment') : 'total_household_payment',
                              ('sum', 'Insurance_payment') : 'total_insurance_payment'          , ('sum', 'Loan_Payment') : 'total_loan_payment',
                              ('sum', 'Other_Order_Payment') : 'total_other_orders'             , ('sum', 'leasing') : 'total_order_leasing'},
                    inplace = True)

In [29]:
order_final = order_final.fillna(0)
order_final.head()


,total_nbr_orders,sum_all_orders,average_all_orders,num_orders_house_payment,num_orders_insurance_payment,num_orders_loan_payment,num_other_orders,num_leasing_payment,avg_household_payment,avg_insurance_payment,avg_loan_payment,avg_other_orders,avg_leasing_payment,total_household_payment,total_insurance_payment,total_loan_payment,total_other_orders,total_order_leasing
account_id,,,,,,,,,,,,,,,,,,
1,1,2452.0,2452.00,1.0,0.0,0.0,0.0,0.0,2452.0,0.0,0.0,0.0,0.0,2452.0,0.0,0.0,0.0,0.0
2,2,10638.7,5319.35,1.0,0.0,1.0,0.0,0.0,7266.0,0.0,3372.7,0.0,0.0,7266.0,0.0,3372.7,0.0,0.0
3,3,5001.0,1667.00,1.0,1.0,0.0,1.0,0.0,1135.0,3539.0,0.0,327.0,0.0,1135.0,3539.0,0.0,327.0,0.0
4,2,3363.0,1681.50,2.0,0.0,0.0,0.0,0.0,1681.5,0.0,0.0,0.0,0.0,3363.0,0.0,0.0,0.0,0.0
5,1,2668.0,2668.00,1.0,0.0,0.0,0.0,0.0,2668.0,0.0,0.0,0.0,0.0,2668.0,0.0,0.0,0.0,0.0


## 8. District Data

In [30]:
explore(district)

   A1           A2               A3       A4  A5  A6  A7  A8  A9    A10  \
0   1  Hl.m. Praha           Prague  1204953   0   0   0   1   1  100.0   
1   2      Benesov  central Bohemia    88884  80  26   6   2   5   46.7   
2   3       Beroun  central Bohemia    75232  55  26   4   1   5   41.7   
3   4       Kladno  central Bohemia   149893  63  29   6   2   6   67.4   
4   5        Kolin  central Bohemia    95616  65  30   4   1   6   51.4   

     A11   A12   A13  A14    A15    A16  
0  12541  0.29  0.43  167  85677  99107  
1   8507  1.67  1.85  132   2159   2674  
2   8980  1.95  2.21  111   2824   2813  
3   9753  4.64  5.05  109   5244   5892  
4   9307  3.85  4.43  118   2616   3040  
    A1               A2             A3      A4  A5  A6  A7  A8  A9    A10  \
72  73            Opava  north Moravia  182027  17  49  12   2   7   56.4   
73  74  Ostrava - mesto  north Moravia  323870   0   0   0   1   1  100.0   
74  75           Prerov  north Moravia  138032  67  30   4   2   5

In [31]:
district.rename(columns={'A1':'district_id'                            , 'A2':'district_name',
                                    'A3':'region'                      , 'A4':'nbr_inhabitants',
                                    'A5':'num_muncipal_<499'           , 'A6':'num_muncipal_500_1999',
                                    'A7':'num_muncipal_2000_9999'      , 'A8':'num_muncipal_>10000',
                                    'A9':'num_cities'                  , 'A10':'urban_ratio',
                                    'A11':'avg_salary'                 , 'A12':'unemp_rate_95', 
                                    'A13': 'unemp_rate_96'             , 'A14':'num_entrepreneurs_per1000',
                                    'A15':'num_crimes_95'              , 'A16':'num_crimes_96'},
                           inplace = True)

district.head()

,district_id,district_name,region,nbr_inhabitants,num_muncipal_<499,num_muncipal_500_1999,num_muncipal_2000_9999,num_muncipal_>10000,num_cities,urban_ratio,avg_salary,unemp_rate_95,unemp_rate_96,num_entrepreneurs_per1000,num_crimes_95,num_crimes_96
0,1,Hl.m. Praha,Prague,1204953,0,0,0,1,1,100.0,12541,0.29,0.43,167,85677,99107
1,2,Benesov,central Bohemia,88884,80,26,6,2,5,46.7,8507,1.67,1.85,132,2159,2674
2,3,Beroun,central Bohemia,75232,55,26,4,1,5,41.7,8980,1.95,2.21,111,2824,2813
3,4,Kladno,central Bohemia,149893,63,29,6,2,6,67.4,9753,4.64,5.05,109,5244,5892
4,5,Kolin,central Bohemia,95616,65,30,4,1,6,51.4,9307,3.85,4.43,118,2616,3040


## 9. Loan Data

### Inspecting the data

In [32]:
explore(loan)

   loan_id  account_id    date  amount  duration  payments status
0     5314        1787  930705   96396        12    8033.0      B
1     5316        1801  930711  165960        36    4610.0      A
2     6863        9188  930728  127080        60    2118.0      A
3     5325        1843  930803  105804        36    2939.0      A
4     7240       11013  930906  274740        60    4579.0      A
     loan_id  account_id    date  amount  duration  payments status
677     4989         105  981205  352704        48    7348.0      C
678     5221        1284  981205   52512        12    4376.0      C
679     6402        6922  981206  139488        24    5812.0      C
680     5346        1928  981206   55632        24    2318.0      C
681     6748        8645  981208  240900        60    4015.0      C
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 682 entries, 0 to 681
Data columns (total 7 columns):
loan_id       682 non-null int64
account_id    682 non-null int64
date          682 non-null

In [33]:
# Changing status observations to a more comprehensible value

loan.loc[loan['status'] == 'A','status'] = 'Finished_Paid'
loan.loc[loan['status'] == 'B','status'] = 'Finished_NotPaid'
loan.loc[loan['status'] == 'C','status'] = 'Running_Okay'
loan.loc[loan['status'] == 'D','status'] = 'Running_InDebt'

# Create dummies for status column

loan = loan.join(get_dummies(loan['status']))
loan = loan.drop(columns=['status'])

# Transform the date 

loan['date'] = loan['date'].transform(lambda yn: '19' + str(yn)[:])
loan['loan_start_date'] =  pd.to_datetime(loan['date'])

# Sorting by account_id
loan.sort_values('account_id')

# View the results
loan.head()

,loan_id,account_id,date,amount,duration,payments,Finished_NotPaid,Finished_Paid,Running_InDebt,Running_Okay,loan_start_date
0,5314,1787,19930705,96396,12,8033.0,1,0,0,0,1993-07-05
1,5316,1801,19930711,165960,36,4610.0,0,1,0,0,1993-07-11
2,6863,9188,19930728,127080,60,2118.0,0,1,0,0,1993-07-28
3,5325,1843,19930803,105804,36,2939.0,0,1,0,0,1993-08-03
4,7240,11013,19930906,274740,60,4579.0,0,1,0,0,1993-09-06


## 10. Client Data

In [34]:
explore(client)

   client_id  birth_number  district_id
0          1        706213           18
1          2        450204            1
2          3        406009            1
3          4        561201            5
4          5        605703            5
      client_id  birth_number  district_id
5364      13955        456030            1
5365      13956        430406            1
5366      13968        680413           61
5367      13971        626019           67
5368      13998        535812           74
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5369 entries, 0 to 5368
Data columns (total 3 columns):
client_id       5369 non-null int64
birth_number    5369 non-null int64
district_id     5369 non-null int64
dtypes: int64(3)
memory usage: 126.0 KB
None
Index(['client_id', 'birth_number', 'district_id'], dtype='object')
client_id       int64
birth_number    int64
district_id     int64
dtype: object
(5369, 3)
          client_id   birth_number  district_id
count   5369.000000    5369.000000  5

In [35]:
# Transform the birth day into year
client['birth_year'] = client['birth_number'].transform(lambda bn: int('19' + str(bn)[:2]))

# Age
client['age'] = 1999 - client['birth_year']

# Age group
client['age_group'] = client['age'] // 10 * 10

# Function to extract birth month and gender
def to_month_gender(birth_number):
    
    # Extract the birth month from the birth_number column
    s = str(birth_number)
    birth_month = int(s[2:4])
    
    # Extract the gender from the birth month
    if birth_month > 50:
        gender = "F"
        birth_month = birth_month - 50
    else:
        gender = 'M'
        
    return pd.Series({'birth_month':birth_month, 'gender':gender})

# Apply the function to client table
client[['birth_month', 'gender']] = client['birth_number'].apply(to_month_gender)

# Drop birth number
client = client.drop(columns = 'birth_number')

#Creating dummies for gender variable
client = pd.get_dummies(client, columns = ['gender'], drop_first=True)

# View the results
client.head()

,client_id,district_id,birth_year,age,age_group,birth_month,gender_M
0,1,18,1970,29,20,12,0
1,2,1,1945,54,50,2,1
2,3,1,1940,59,50,10,0
3,4,5,1956,43,40,12,1
4,5,5,1960,39,30,7,0


## 11. Disposition Data

In [36]:
explore(disp)

   disp_id  client_id  account_id       type
0        1          1           1      OWNER
1        2          2           2      OWNER
2        3          3           2  DISPONENT
3        4          4           3      OWNER
4        5          5           3  DISPONENT
      disp_id  client_id  account_id       type
5364    13647      13955       11349      OWNER
5365    13648      13956       11349  DISPONENT
5366    13660      13968       11359      OWNER
5367    13663      13971       11362      OWNER
5368    13690      13998       11382      OWNER
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5369 entries, 0 to 5368
Data columns (total 4 columns):
disp_id       5369 non-null int64
client_id     5369 non-null int64
account_id    5369 non-null int64
type          5369 non-null object
dtypes: int64(3), object(1)
memory usage: 167.9+ KB
None
Index(['disp_id', 'client_id', 'account_id', 'type'], dtype='object')
disp_id        int64
client_id      int64
account_id     int64
type     

In [37]:
# Create dummies for disposition type
disp = disp.join(pd.get_dummies(disp["type"], prefix = "Is_", drop_first = True))

disp = disp.drop(columns=['type'])

disp.head()

,disp_id,client_id,account_id,Is__OWNER
0,1,1,1,1
1,2,2,2,1
2,3,3,2,0
3,4,4,3,1
4,5,5,3,0


## 12. Merging all Data Sets

In [38]:
#Merging Disposition and client dataset
disp_client = pd.merge(disp, client, 'left', on = 'client_id')

#Adding account dataset to the merged data
disp_client_acc = pd.merge(disp_client, account, 'left', on= ['account_id', 'district_id'])

#Adding district dataset to the merged data
disp_client_acc_dist = pd.merge(disp_client_acc, district, 'left', on= 'district_id')

#Adding transaction dataset to the merged data
disp_client_acc_dist_trans = pd.merge(disp_client_acc_dist, trans_final, 'left', on = 'account_id')

#Adding orders to the merged dataset 
disp_client_acc_dist_trans_orders = pd.merge(disp_client_acc_dist_trans, order_final, 'left', on = 'account_id')

#Adding loans to the merged dataset 
disp_client_acc_dist_trans_orders_loan = pd.merge(disp_client_acc_dist_trans_orders, loan, 'left', on = 'account_id')

#Adding credit card dataset to the merged dataset 
data_mart = pd.merge(disp_client_acc_dist_trans_orders_loan, card, 'left', on = 'disp_id')

# Change NA for 0 
data_mart = data_mart.fillna(0)

#View the final result
data_mart.head()

,disp_id,client_id,account_id,Is__OWNER,district_id,birth_year,age,age_group,birth_month,gender_M,...,duration,payments,Finished_NotPaid,Finished_Paid,Running_InDebt,Running_Okay,loan_start_date,card_id,type,issued
0,1,1,1,1,18,1970,29,20,12,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0
1,2,2,2,1,1,1945,54,50,2,1,...,24.0,3373.0,0.0,1.0,0.0,0.0,1994-01-05 00:00:00,0.0,0,0
2,3,3,2,0,1,1940,59,50,10,0,...,24.0,3373.0,0.0,1.0,0.0,0.0,1994-01-05 00:00:00,0.0,0,0
3,4,4,3,1,5,1956,43,40,12,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0
4,5,5,3,0,5,1960,39,30,7,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0


 ## 13. Segmentation

### Standardising the variables

In [39]:
#Get column names 
names = ["age", "age_group", "avg_salary", "days_since_acc_creation", "total_trans",
         "mean_all_trans", "days_since_last_trans", "total_nbr_orders", "average_all_orders"]

#Winsorising variables to treat outliers before segmentation
for col in names:
    data_mart[col] = winsorize(array(data_mart[col]), limits=[0.00, 0.01])


In [40]:
#Create the scaler object
scaler = preprocessing.StandardScaler()

#Fitting the data on the scaler object
data_mart[names] = scaler.fit_transform(data_mart[names])

In [41]:
#KMeans Clustering 
data= data_mart

#Fitting the model 
kmeans = KMeans(n_clusters = 3)
data["kmean cluster"] = kmeans.fit_predict(data[names])

data["kmean cluster"].head()

0    2
1    2
2    1
3    2
4    2
Name: kmean cluster, dtype: int32

In [42]:
#Reversing the standardised scale to interpret the results
data[names] = scaler.inverse_transform(data[names])
results = pd.DataFrame.from_records(scaler.inverse_transform(kmeans.cluster_centers_), columns=names).round(2)
results

,age,age_group,avg_salary,days_since_acc_creation,total_trans,mean_all_trans,days_since_last_trans,total_nbr_orders,average_all_orders
0,38.42,34.28,9452.30,1034.18,113.30,4951.10,29.06,0.31,1244.36
1,63.10,58.73,9472.09,1040.21,218.15,3912.54,1.01,1.44,2743.94
2,34.41,29.87,9540.90,1072.42,253.78,6926.45,1.03,1.56,3490.39


## Exporting Results

In [43]:
results.to_csv("C:/Users/bvarghese/Desktop/IESEG Slides/Financial Programming- Python- Phan Minh/Group Project_Customer Loan/Results_Final_group_6.csv")

In [44]:
data_mart.to_csv("C:/Users/bvarghese/Desktop/IESEG Slides/Financial Programming- Python- Phan Minh/Group Project_Customer Loan/data_mart_group_6.csv")